# Text Classification with NLP and Word Embeddings
**Objective:** Implement an NLP pipeline using TF-IDF and Word2Vec embeddings with classifiers.

## Step 1: Install & Import Libraries

In [ ]:

# !pip install numpy pandas scikit-learn matplotlib nltk gensim joblib tensorflow streamlit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from gensim.models import Word2Vec
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


## Step 2: Load Dataset

In [ ]:

# Using SMS Spam Dataset
url = 'https://raw.githubusercontent.com/justmarkham/scikit-learn-videos/master/data/sms.tsv'
df = pd.read_csv(url, sep='\t')
df.columns = ['label', 'text']
df.head()


## Step 3: Preprocessing

In [ ]:

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess(text):
    text = text.lower()
    text = re.sub(r'[^a-z ]', '', text)
    tokens = nltk.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return ' '.join(tokens)

df['clean_text'] = df['text'].apply(preprocess)
df.head()


## Step 4: Train/Test Split

In [ ]:

X = df['clean_text']
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Step 5: TF-IDF + Logistic Regression & Naive Bayes

In [ ]:

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

# Logistic Regression
lr = LogisticRegression(max_iter=200)
lr.fit(X_train_tfidf, y_train)
y_pred_lr = lr.predict(X_test_tfidf)

# Naive Bayes
nb = MultinomialNB()
nb.fit(X_train_tfidf, y_train)
y_pred_nb = nb.predict(X_test_tfidf)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("Naive Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print("\nClassification Report for Logistic Regression:\n", classification_report(y_test, y_pred_lr))
print("\nClassification Report for Naive Bayes:\n", classification_report(y_test, y_pred_nb))


## Step 6: Word2Vec Embeddings + Random Forest

In [ ]:

# Tokenize sentences
sentences = [nltk.word_tokenize(text) for text in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

def vectorize(text, model, size=100):
    words = nltk.word_tokenize(text)
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in words:
        if word in model.wv:
            vec += model.wv[word].reshape((1, size))
            count += 1.
    if count != 0:
        vec /= count
    return vec

X_train_w2v = np.concatenate([vectorize(text, w2v_model) for text in X_train])
X_test_w2v = np.concatenate([vectorize(text, w2v_model) for text in X_test])

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train_w2v, y_train)
y_pred_rf = rf.predict(X_test_w2v)

print("Random Forest with Word2Vec Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report for Random Forest:\n", classification_report(y_test, y_pred_rf))


## Step 7: Confusion Matrices

In [ ]:

import seaborn as sns

cm_lr = confusion_matrix(y_test, y_pred_lr)
cm_nb = confusion_matrix(y_test, y_pred_nb)
cm_rf = confusion_matrix(y_test, y_pred_rf)

fig, axs = plt.subplots(1, 3, figsize=(15,5))
sns.heatmap(cm_lr, annot=True, fmt='d', ax=axs[0], cmap='Blues')
axs[0].set_title('Logistic Regression')

sns.heatmap(cm_nb, annot=True, fmt='d', ax=axs[1], cmap='Greens')
axs[1].set_title('Naive Bayes')

sns.heatmap(cm_rf, annot=True, fmt='d', ax=axs[2], cmap='Oranges')
axs[2].set_title('Random Forest')

plt.show()
